# Traffic Sign Classification with Keras

Keras exists to make coding deep neural networks simpler. To demonstrate just how easy it is, you’re going to use Keras to build a convolutional neural network in a few dozen lines of code.

You’ll be connecting the concepts from the previous lessons to the methods that Keras provides.

## Dataset

The network you'll build with Keras is similar to the example that you can find in Keras’s GitHub repository that builds out a [convolutional neural network for MNIST](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py). 

However, instead of using the [MNIST](http://yann.lecun.com/exdb/mnist/) dataset, you're going to use the [German Traffic Sign Recognition Benchmark](http://benchmark.ini.rub.de/?section=gtsrb&subsection=news) dataset that you've used previously.

You can download pickle files with sanitized traffic sign data here.

## Overview

Here are the steps you'll take to build the network:

1. First load the training data and do a train/validation split.
2. Preprocess data.
3. Build a feedforward neural network to classify traffic signs.
4. Build a convolutional neural network to classify traffic signs.
5. Evaluate performance of final neural network on testing data.

Keep an eye on the network’s accuracy over time. Once the accuracy reaches the 98% range, you can be confident that you’ve built and trained an effective model.

In [8]:
import pickle
import numpy as np
from sklearn.cross_validation import train_test_split
import math
import cv2
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils
from keras.layers import Convolution2D, MaxPooling2D, Flatten


## Load the Data

Start by importing the data from the pickle file.

In [2]:
# TODO: Implement load the data here.
with open('./Data/train.p', 'rb') as f:
    data = pickle.load(f)

## Validate the Network
Split the training data into a training and validation set.

Measure the [validation accuracy](https://keras.io/models/sequential/) of the network after two training epochs.

Hint: [Use the `train_test_split()` method](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) from scikit-learn.

In [3]:
# TODO: Use `train_test_split` here.
pool_size = (2, 2)
nb_classes = 43
batch_size = 512
nb_epoch = 50
input_shape = (32, 32, 3)

X_train = data['features']
y_train = data['labels']

X_train, X_val, y_train, y_val  = train_test_split(X_train, y_train, test_size=0.05)

In [4]:
# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert(X_train.shape[0] == y_train.shape[0]), "The number of images is not equal to the number of labels."
assert(X_train.shape[1:] == (32,32,3)), "The dimensions of the images are not 32 x 32 x 3."
assert(X_val.shape[0] == y_val.shape[0]), "The number of images is not equal to the number of labels."
assert(X_val.shape[1:] == (32,32,3)), "The dimensions of the images are not 32 x 32 x 3."

## Preprocess the Data

Now that you've loaded the training data, preprocess the data such that it's in the range between -0.5 and 0.5.

In [5]:
# TODO: Implement data normalization here.

size = np.shape(X_train)[0]
X_train = ((X_train - np.min(X_train))/(np.max(X_train) - np.min(X_train))) - 0.5
X_val = ((X_val - np.min(X_val))/(np.max(X_val) - np.min(X_val))) - 0.5

In [6]:
# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert(math.isclose(np.min(X_train), -0.5, abs_tol=1e-5) and math.isclose(np.max(X_train), 0.5, abs_tol=1e-5)), "The range of the training data is: %.1f to %.1f" % (np.min(X_train), np.max(X_train))
assert(math.isclose(np.min(X_val), -0.5, abs_tol=1e-5) and math.isclose(np.max(X_val), 0.5, abs_tol=1e-5)), "The range of the validation data is: %.1f to %.1f" % (np.min(X_val), np.max(X_val))

## Build a Two-Layer Feedfoward Network

The code you've written so far is for data processing, not specific to Keras. Here you're going to build Keras-specific code.

Build a two-layer feedforward neural network, with 128 neurons in the fully-connected hidden layer. 

To get started, review the Keras documentation about [models](https://keras.io/models/sequential/) and [layers](https://keras.io/layers/core/).

The Keras example of a [Multi-Layer Perceptron](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py) network is similar to what you need to do here. Use that as a guide, but keep in mind that there are a number of differences.

In [ ]:
# TODO: Build a two-layer feedforward neural network with Keras here.

model = Sequential()
model.add(Dense(128, input_shape=(3072,)))
model.add(Activation('relu'))
model.add(Dense(43, activation='softmax'))
model.compile(optimizer='rmsprop',
          loss='categorical_crossentropy',
          metrics=['accuracy'])

In [ ]:
# STOP: Do not change the tests below. Your implementation should pass these tests.
dense_layers = []
for l in model.layers:
    if type(l) == Dense:
        dense_layers.append(l)
assert(len(dense_layers) == 2), "There should be 2 Dense layers."
d1 = dense_layers[0]
d2 = dense_layers[1]
assert(d1.input_shape == (None, 3072))
assert(d1.output_shape == (None, 128))
assert(d2.input_shape == (None, 128))
assert(d2.output_shape == (None, 43))

last_layer = model.layers[-1]
assert(last_layer.activation.__name__ == 'softmax'), "Last layer should be softmax activation, is {}.".format(last_layer.activation.__name__)

In [ ]:
# Debugging
for l in model.layers:
    print(l.name, l.input_shape, l.output_shape, l.activation)

## Train the Network
Compile and train the network for 2 epochs. [Use the `adam` optimizer, with `categorical_crossentropy` loss.](https://keras.io/models/sequential/)

Hint 1: In order to use categorical cross entropy, you will need to [one-hot encode the labels](https://github.com/fchollet/keras/blob/master/keras/utils/np_utils.py).

Hint 2: In order to pass the input images to the fully-connected hidden layer, you will need to [reshape the input](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py).

Hint 3: Keras's `.fit()` method returns a `History.history` object, which the tests below use. Save that to a variable named `history`.

In [13]:
# TODO: Compile and train the model here.

# One hot encode
y_train = np_utils.to_categorical(y_train, nb_classes)
y_val= np_utils.to_categorical(y_val, nb_classes)
X_train = X_train.reshape(X_train.shape[0], 3072)
X_val = X_val.reshape(X_val.shape[0], 3072)


In [14]:
history = model.fit(X_train, y_train,nb_epoch=nb_epoch,verbose=1, validation_data=(X_val, y_val))

NameError: name 'model' is not defined

In [15]:
# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['acc'][-1] > 0.92), "The training accuracy was: %.3f" % history.history['acc'][-1]
assert(history.history['val_acc'][-1] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][-1]

NameError: name 'history' is not defined

**Validation Accuracy**: 0.9393

## Congratulations
You've built a feedforward neural network in Keras!

Don't stop here! Next, you'll add a convolutional layer to drive.py.

## Convolutions
Build a new network, similar to your existing network. Before the hidden layer, add a 3x3 [convolutional layer](https://keras.io/layers/convolutional/#convolution2d) with 32 filters and valid padding.

Then compile and train the network.

Hint 1: The Keras example of a [convolutional neural network](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py) for MNIST would be a good example to review.

Hint 2: Now that the first layer of the network is a convolutional layer, you no longer need to reshape the input images before passing them to the network. You might need to reload your training data to recover the original shape.

Hint 3: Add a [`Flatten()` layer](https://keras.io/layers/core/#flatten) between the convolutional layer and the fully-connected hidden layer.

In [ ]:
# TODO: Re-construct the network and add a convolutional layer before the first fully-connected layer.


X_train = X_train.reshape(X_train.shape[0], 32, 32, 3)
X_val = X_val.reshape(X_val.shape[0], 32, 32, 3)

model = Sequential()
model.add(Convolution2D(32, 3, 3,
                        border_mode='valid',
                        input_shape=input_shape))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(43, activation='softmax'))
model.compile(optimizer='rmsprop',
          loss='categorical_crossentropy',
          metrics=['accuracy'])

history = model.fit(X_train, y_train,nb_epoch=nb_epoch,verbose=1, validation_data=(X_val, y_val))

In [ ]:
# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][-1] > 0.95), "The validation accuracy is: %.3f" % history.history['val_acc'][-1]

**Validation Accuracy**: 0.972463029067

## Pooling
Re-construct your network and add a 2x2 [pooling layer](https://keras.io/layers/pooling/#maxpooling2d) immediately following your convolutional layer.

Then compile and train the network.

In [ ]:
# TODO: Re-construct the network and add a pooling layer after the convolutional layer.
pool_size = (2, 2)

model = Sequential()
model.add(Convolution2D(32, 3, 3,
                        border_mode='valid',
                        input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(43, activation='softmax'))
model.compile(optimizer='rmsprop',
          loss='categorical_crossentropy',
          metrics=['accuracy'])

history = model.fit(X_train, y_train,nb_epoch=nb_epoch,verbose=1, validation_data=(X_val, y_val))

In [ ]:
# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][-1] > 0.95), "The validation accuracy is: %.3f" % history.history['val_acc'][-1]

**Validation Accuracy**: 0.9837

## Dropout
Re-construct your network and add [dropout](https://keras.io/layers/core/#dropout) after the pooling layer. Set the dropout rate to 50%.

In [ ]:
# TODO: Re-construct the network and add dropout after the pooling layer.


model = Sequential()
model.add(Convolution2D(32, 3, 3,
                        border_mode='valid',
                        input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(43, activation='softmax'))
model.compile(optimizer='rmsprop',
          loss='categorical_crossentropy',
          metrics=['accuracy'])

history = model.fit(X_train, y_train,nb_epoch=nb_epoch,verbose=1, validation_data=(X_val, y_val))

In [ ]:
# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][-1] > 0.95), "The validation accuracy is: %.3f" % history.history['val_acc'][-1]

**Validation Accuracy**: 0.9898

## Optimization
Congratulations! You've built a neural network with convolutions, pooling, dropout, and fully-connected layers, all in just a few lines of code.

Have fun with the model and see how well you can do! Add more layers, or regularization, or different padding, or batches, or more training epochs.

What is the best validation accuracy you can achieve?

In [16]:
input_shape = (32, 32, 3)

X_train = X_train.reshape(X_train.shape[0], 32, 32, 3)
X_val = X_val.reshape(X_val.shape[0], 32, 32, 3)


model2 = Sequential()
model2.add(Convolution2D(32, 3, 3,
                        border_mode='valid',
                        input_shape=input_shape))
model2.add(Activation('relu'))
model2.add(MaxPooling2D(pool_size=pool_size))
model2.add(Convolution2D(16, 3, 3,
                        border_mode='valid',
                        input_shape=input_shape))
model2.add(Convolution2D(32, 5, 5,
                        border_mode='same',
                        input_shape=input_shape))
model2.add(Convolution2D(64, 1, 1,
                        border_mode='same',
                        input_shape=input_shape))

model2.add(Dropout(0.5))
model2.add(Flatten())
model2.add(Dense(512))
model2.add(Activation('relu'))
model2.add(Dense(43, activation='softmax'))

model2.compile(optimizer='rmsprop',
          loss='categorical_crossentropy',
          metrics=['accuracy'])

history = model2.fit(X_train, y_train, batch_size=batch_size, nb_epoch=nb_epoch,verbose=1, validation_data=(X_val, y_val))


Train on 37248 samples, validate on 1961 samples
Epoch 1/50
37248/37248 [==============================] - 54s - loss: 2.4537 - acc: 0.3357 - val_loss: 1.0805 - val_acc: 0.6808
Epoch 2/50
37248/37248 [==============================] - 56s - loss: 0.6192 - acc: 0.8129 - val_loss: 0.3733 - val_acc: 0.8914
Epoch 3/50
37248/37248 [==============================] - 55s - loss: 0.2496 - acc: 0.9293 - val_loss: 0.2461 - val_acc: 0.9434
Epoch 4/50
37248/37248 [==============================] - 55s - loss: 0.1545 - acc: 0.9566 - val_loss: 0.1625 - val_acc: 0.9628
Epoch 5/50
37248/37248 [==============================] - 53s - loss: 0.0973 - acc: 0.9726 - val_loss: 0.1140 - val_acc: 0.9760
Epoch 6/50
37248/37248 [==============================] - 53s - loss: 0.0746 - acc: 0.9789 - val_loss: 0.1379 - val_acc: 0.9725
Epoch 7/50
37248/37248 [==============================] - 55s - loss: 0.0485 - acc: 0.9863 - val_loss: 0.1479 - val_acc: 0.9735
Epoch 8/50
37248/37248 [==============================]

**Best Validation Accuracy:** 0.9878

## Testing
Once you've picked out your best model, it's time to test it.

Load up the test data and use the [`evaluate()` method](https://keras.io/models/model/#evaluate) to see how well it does.

Hint 1: The `evaluate()` method should return an array of numbers. Use the `metrics_names()` method to get the labels.

In [20]:
# TODO: Load test data
with open('./Data/test.p', 'rb') as f:
    test = pickle.load(f)
X_test = test['features']
y_test = test['labels']    

# TODO: Preprocess data & one-hot encode the labels
y_test= np_utils.to_categorical(y_test, nb_classes)
X_test = ((X_test - np.min(X_test))/(np.max(X_test) - np.min(X_test))) - 0.5
X_test = X_test.reshape(X_test.shape[0], 32, 32, 3)


# TODO: Evaluate model on test data
# m = model2
model2.evaluate(X_test, y_test)

12630/12630 [==============================] - 7s     


[0.71403611556445679, 0.92897862236554529]

**Test Accuracy:** 0.9289

## Summary
Keras is a great tool to use if you want to quickly build a neural network and evaluate performance.